In [ ]:
from Preprocessing.Preprocessor import Preprocessor
from models.generic_model import Model
from data_utils.keras_generator_stratified import Generator
from utils.losses import weighted_cross_entropy_loss
from utils.callbacks import get_tensorboard_callback
from keras import optimizers
import os
import pandas as pd


In [ ]:
label_encoder_dir = "Preprocessing"
train_data_dir = r'D:\Val_Data'


In [ ]:
train_files = os.listdir(train_data_dir)
array=pd.read_hdf(os.path.join(train_data_dir, train_files[0]))
P = Preprocessor(label_encoder_dir)
array = P.process_batch(array)
action_col=array.columns == 'action'
g_config={'labels': 6,
          'weightage': [1, 1, 1, 1, 1, 1],
          'data_directory': train_data_dir,
          'file_batch_size': 1500,
          'format_': 'h5',
          'ex_batch_size': 10000,
          'file_examples': 5000,
          'max_queue_size': 1,
          'Preprocessor': P,
          'static_weightage': False,
          'action_col': action_col}

G = Generator(config=g_config)


In [ ]:
nn_config = {'layer_shapes': [500, 500, 500], 'Activation': 'relu',
             'Output': 6, 'Input_shape': 472, 'model_type': 'fc'}
my_model = Model(nn_config)
my_model.model.compile(loss=weighted_cross_entropy_loss, optimizer=optimizers.Adam(lr=5e-4), metrics=["acc"])


In [ ]:
G.fill_buffer()


my_model.model.summary()

In [ ]:
my_model.model.fit_generator(generator=G, steps_per_epoch=len(G), epochs=300, verbose=1, max_queue_size=3)
